### Описание данных

- id — уникальный идентификатор скважины;
- f0
- f1
- f2 — три признака точек;
- product — объём запасов в скважине (тыс. баррелей).


# 1. Загрузка и подготовка данных

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
first_region.head()
second_region.head()
third_region.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [4]:
first_region.info()
second_region.info()
third_region.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8

**Пропусков** нет, названия столбцов в английской раскладке, в нижнем регистре. Лишних столбцов нет.

**Посмотрим дубликаты**

In [5]:
print('Количество дубликатов по id в первом регионе:', int(pd.DataFrame(first_region['id'].value_counts()).query('id > 1').count()))
print('Количество дубликатов по id во втором регионе:', int(pd.DataFrame(second_region['id'].value_counts()).query('id > 1').count()))
print('Количество дубликатов по id во третьем регионе:', int(pd.DataFrame(third_region['id'].value_counts()).query('id > 1').count()))

Количество дубликатов по id в первом регионе: 10
Количество дубликатов по id во втором регионе: 4
Количество дубликатов по id во третьем регионе: 4


In [6]:
print('Количество дубликатов по 1 признаку в первом регионе:', int(pd.DataFrame(first_region['f0'].value_counts()).query('f0 > 1').count()))
print('Количество дубликатов по 1 признаку во втором регионе:', int(pd.DataFrame(second_region['f0'].value_counts()).query('f0 > 1').count()))
print('Количество дубликатов по 1 признаку во третьем регионе:', int(pd.DataFrame(third_region['f0'].value_counts()).query('f0 > 1').count()))

Количество дубликатов по 1 признаку в первом регионе: 0
Количество дубликатов по 1 признаку во втором регионе: 0
Количество дубликатов по 1 признаку во третьем регионе: 0


In [7]:
print('Количество дубликатов по 2 признаку в первом регионе:', int(pd.DataFrame(first_region['f1'].value_counts()).query('f1 > 1').count()))
print('Количество дубликатов по 2 признаку во втором регионе:', int(pd.DataFrame(second_region['f1'].value_counts()).query('f1 > 1').count()))
print('Количество дубликатов по 2 признаку во третьем регионе:', int(pd.DataFrame(third_region['f1'].value_counts()).query('f1 > 1').count()))

Количество дубликатов по 2 признаку в первом регионе: 0
Количество дубликатов по 2 признаку во втором регионе: 0
Количество дубликатов по 2 признаку во третьем регионе: 0


In [8]:
print('Количество дубликатов по 3 признаку в первом регионе:', int(pd.DataFrame(first_region['f2'].value_counts()).query('f2 > 1').count()))
print('Количество дубликатов по 3 признаку во втором регионе:', int(pd.DataFrame(second_region['f2'].value_counts()).query('f2 > 1').count()))
print('Количество дубликатов по 3 признаку во третьем регионе:', int(pd.DataFrame(third_region['f2'].value_counts()).query('f2 > 1').count()))

Количество дубликатов по 3 признаку в первом регионе: 0
Количество дубликатов по 3 признаку во втором регионе: 0
Количество дубликатов по 3 признаку во третьем регионе: 0


С одной стороны, показатели для каждой продублированной скважины можно сложить - тогда объём запасов в них просуммируется, это относительно логично. С другой стороны, в таком случае каждую скважину проанализировали и нашли сначала одну половину запасов, потом другую - что невозможно. Суммировать 3 остальные характеристики также некорректно, так как мы не знаем их природу и метод анализа скважины. Так как таких данных мало, их можно просто удалить.

In [9]:
first_region = first_region.drop_duplicates(subset='id')
second_region = second_region.drop_duplicates(subset='id')
third_region = third_region.drop_duplicates(subset='id')

В самих таблицах айдишники нужны для предобработки данных, а в процессе обучения модели будут мешаться, поэтому айдишники во всех трёх регионах удалим

In [10]:
del first_region['id']
del second_region['id']
del third_region['id']

# 2. Обучение и проверка модели

**Регион 1**

In [11]:
first_train, first_valid = train_test_split(first_region, test_size=0.25, random_state=12345)

first_target_train = first_train['product'].reset_index(drop=True)
first_features_train = first_train.copy(deep=True).reset_index(drop=True)
del first_features_train['product']

first_target_valid = first_valid['product'].reset_index(drop=True).reset_index(drop=True)
first_features_valid = first_valid.copy(deep=True).reset_index(drop=True)
del first_features_valid['product']

In [12]:
first_model = LinearRegression().fit(first_features_train, first_target_train)
first_model_predict = pd.Series(first_model.predict(first_features_valid))
print('rmse =', mean_squared_error(first_model_predict, first_target_valid) ** 0.5)
print('Средний запас предсказанного сырья =',first_model_predict.mean())

rmse = 37.853527328872964
Средний запас предсказанного сырья = 92.78915638280621


**Регион 2**

In [13]:
second_train, second_valid = train_test_split(second_region, test_size=0.25, random_state=12345)

second_target_train = second_train['product'].reset_index(drop=True)
second_features_train = second_train.copy(deep=True).reset_index(drop=True)
del second_features_train['product']

second_target_valid = second_valid['product'].reset_index(drop=True)
second_features_valid = second_valid.copy(deep=True).reset_index(drop=True)
del second_features_valid['product']

In [14]:
second_model = LinearRegression().fit(second_features_train, second_target_train)
second_model_predict = pd.Series(second_model.predict(second_features_valid))
print('rmse =', mean_squared_error(second_model_predict, second_target_valid) ** 0.5)
print('Средний запас предсказанного сырья =',second_model_predict.mean())

rmse = 0.8920592647717019
Средний запас предсказанного сырья = 69.17831957030432


**Регион 3**

In [15]:
third_train, third_valid = train_test_split(third_region, test_size=0.25, random_state=12345)

third_target_train = third_train['product'].reset_index(drop=True)
third_features_train = third_train.copy(deep=True).reset_index(drop=True)
del third_features_train['product']

third_target_valid = third_valid['product']
third_features_valid = third_valid.copy(deep=True)
del third_features_valid['product']

In [16]:
third_model = LinearRegression().fit(third_features_train, third_target_train)
third_model_predict = pd.Series(third_model.predict(third_features_valid))
print('rmse =', mean_squared_error(third_model_predict, third_target_valid) ** 0.5)
print('Средний запас предсказанного сырья =',third_model_predict.mean())

rmse = 40.07585073246016
Средний запас предсказанного сырья = 94.86572480562035


Первый и третий регионы понятны модели хуже, чем второй, однако в них предсказано сырья в 1,32 раза больше.

# 3. Подготовка к расчёту прибыли

In [17]:
BUDGET = 10 ** 10
REVENUE_PER_BARREL = 450 * 10**3
WELLS_NUMBER = 200
ANALYZED_WELLS = 500
DIFFERENT_SAMPLES_NUMBER = 1000
print('Достаточный объём сырья для безубыточной разработки одной новой скважины из предполагаемых 200сот =', BUDGET / WELLS_NUMBER / REVENUE_PER_BARREL)

Достаточный объём сырья для безубыточной разработки одной новой скважины из предполагаемых 200сот = 111.11111111111111


Среднего запаса сырья в каждом регионе не хватает. Именно поэтому в данном случае следует использовать машинное обучение и находить те скважины, в которых запаса больше

# 4. Расчёт прибыли

In [35]:
def revenue(target, predict, count):
    best = pd.Series(predict).sort_values(ascending=False)
    selected = target[best.index][:count]
    return (REVENUE_PER_BARREL * selected.sum()) - BUDGET

print('---Первый регион---')
print('Прибыль =', revenue(first_target_valid, first_model_predict, len(first_model_predict)))
print()
print('---Второй регион---')
print('Прибыль =', revenue(second_target_valid, second_model_predict, len(second_model_predict)))
print()
print('---Третий регион---')
print('Прибыль =', revenue(third_target_valid, third_model_predict, len(third_model_predict)))

---Первый регион---
Прибыль = 1026696862846.3364

---Второй регион---
Прибыль = 768311861387.9342

---Третий регион---
Прибыль = 251976289796.1191


# 5. Расчёт  рисков 

**Первый регион**

In [42]:
state = np.random.RandomState(12345)
    
values = []
for i in range(DIFFERENT_SAMPLES_NUMBER):
    target_subsample = first_target_valid.sample(n=ANALYZED_WELLS, replace=True, random_state=state)
    probs_subsample = first_model_predict[target_subsample.index]
    rev = revenue(target_subsample, probs_subsample, WELLS_NUMBER)
    values.append(rev)

values = pd.Series(values)
confidence_interval = (values.quantile(0.025), values.quantile(0.975))
risk_of_loss = len(values[values < 0]) * 100 / len(values)
mean = values.mean()
print("Средняя выручка:", mean)
print("95%-ый доверительный интервал:", confidence_interval)
print('Риск убытка =', risk_of_loss, "%")

Средняя выручка: 409428038.62143606
95%-ый доверительный интервал: (-131536028.70166382, 944395582.7546725)
Риск убытка = 7.1 %


**Второй регион**

In [45]:
values = []
for i in range(DIFFERENT_SAMPLES_NUMBER):
    target_subsample = second_target_valid.sample(n=ANALYZED_WELLS, replace=True, random_state=state)
    probs_subsample = second_model_predict[target_subsample.index]
    rev = revenue(target_subsample, probs_subsample, WELLS_NUMBER)
    values.append(rev)

values = pd.Series(values)
confidence_interval = (values.quantile(0.025), values.quantile(0.975))
risk_of_loss = len(values[values < 0]) * 100 / len(values)
mean = values.mean()
print("Средняя выручка:", mean)
print("95%-ый доверительный интервал:", confidence_interval)
print('Риск убытка =', risk_of_loss, '%')

Средняя выручка: 527685754.0114966
95%-ый доверительный интервал: (129535167.64558092, 956581469.1039761)
Риск убытка = 0.4 %


**Третий регион**

In [46]:
values = []
for i in range(DIFFERENT_SAMPLES_NUMBER):
    target_subsample = third_target_valid.sample(n=ANALYZED_WELLS, replace=True, random_state=state)
    probs_subsample = third_model_predict[target_subsample.index]
    rev = revenue(target_subsample, probs_subsample, WELLS_NUMBER)
    values.append(rev)

values = pd.Series(values)
confidence_interval = (values.quantile(0.025), values.quantile(0.975))
risk_of_loss = len(values[values < 0]) * 100 / len(values)
mean = values.mean()
print("Средняя выручка:", mean)
print("95%-ый доверительный интервал:", confidence_interval)
print('Риск убытка =', risk_of_loss, '%')

Средняя выручка: -1476346130.4266067
95%-ый доверительный интервал: (-2094147516.8253927, -886324753.0472877)
Риск убытка = 100.0 %


### Вывод по всей работе.

    Во-первых, стоит отметить, что модель совершенно точно научилась прогнозировать количество сырья во втором регионе и научилась чуть хуже в первом и третьем. Если оценить rmse в 40 деньгами, можем потерять 18 000 000 или наоборот внезапно их заработать.
    Во-вторых, прибыль в первом регионе может быть в 1,4 раза больше чем во-втором, а в третьем просто отрицательна.
    В-третьих, риски убытка в первом регионе 7%, во втором 0,4%, а в третьем 100%.
    
Подытожив, выбирать стоит из первых двух регионов. В первом больше прибыль, но больше риски и меньше вероятность в будущем построить новые модели для дальнейших решений где бурить новые скважины в регионе. Во втором меньше выручка, но он более предсказуем. На мой взгляд второй регион больше подходит при таких исходных данных.